In [9]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

DATA_PATH = "../../Datasets/MAMS-ATSA/Original/train/train.xml"

OUTPUT_SINGLE_ROW = "../../Datasets/MAMS-ATSA/Downsampled/train/train_single_row.csv"
OUTPUT_MULTI_ROW = "../../Datasets/MAMS-ATSA/Downsampled/train/train_multi_row.csv"




In [10]:
import pandas as pd
import os
import xml.etree.ElementTree as ET

#Imports the XML file and returns a DataFrame
def parse_xml_to_dataframe(xml_file_path):
    tree = ET.parse(xml_file_path)
    root = tree.getroot()

    texts = []
    aspect_terms = []

    for sentence in root.findall('./sentence'):
        text = sentence.find('text').text
        from_indices = []
        to_indices = []
        polarities = []
        terms = []

        for aspect_term in sentence.findall('./aspectTerms/aspectTerm'):
            from_index = int(aspect_term.get('from'))
            to_index = int(aspect_term.get('to'))
            polarity = aspect_term.get('polarity')
            term = aspect_term.get('term')

            from_indices.append(from_index)
            to_indices.append(to_index)
            polarities.append(polarity)
            terms.append(term)

        texts.append(text)
        aspect_terms.append({'from': from_indices, 'to': to_indices, 'polarity': polarities, 'term': terms})

    # Create a DataFrame
    df = pd.DataFrame({'text': texts, 'aspect_terms': aspect_terms})
    return df
# Convert XML to DataFrame
df = parse_xml_to_dataframe(DATA_PATH)
df_samples = df
# Print the DataFrame 
print(df.columns)
print(df.head())


Index(['text', 'aspect_terms'], dtype='object')
                                                text  \
0  The decor is not special at all but their food...   
1  when tables opened up, the manager sat another...   
2  Though the menu includes some unorthodox offer...   
3  service is good although a bit in your face, w...   
4  PS- I just went for brunch on Saturday and the...   

                                        aspect_terms  
0  {'from': [4, 42, 59], 'to': [9, 46, 65], 'pola...  
1  {'from': [5, 27], 'to': [11, 34], 'polarity': ...  
2  {'from': [11, 54, 93, 145], 'to': [15, 72, 101...  
3  {'from': [0, 78], 'to': [7, 82], 'polarity': [...  
4  {'from': [20, 47], 'to': [26, 70], 'polarity':...  


In [12]:
#Transform DF
def transformDF(df):
    text_id = []
    term_id = []
    texts = []
    terms = []
    polarities = []
    x = 0  

    for i in range(0, len(df)):
        for j in range(0, len(df["aspect_terms"][i]["term"])):
            text_id.append(i)
            term_id.append(x)
            texts.append(df["text"][i])
            terms.append(df["aspect_terms"][i]["term"][j])
            polarities.append(df["aspect_terms"][i]["polarity"][j])
            x += 1

    df_transformed = pd.DataFrame({'text_id' : text_id, "term_id" :term_id, 'text' : texts, 'term' : terms, 'polarity' : polarities})
    return df_transformed

df_transformed = transformDF(df_samples)

print(len(df_transformed))




11186


In [13]:
def print_column_counts(df, column_name):
    polarity_counts = df[column_name].value_counts()
    print(polarity_counts)
    percentage = polarity_counts / polarity_counts.sum() * 100
    print(percentage)
print(len(df_transformed))
print_column_counts(df_transformed, 'polarity')


11186
polarity
neutral     5042
positive    3380
negative    2764
Name: count, dtype: int64
polarity
neutral     45.074200
positive    30.216342
negative    24.709458
Name: count, dtype: float64


In [8]:
df_samples.to_csv(OUTPUT_MULTI_ROW, index=False)
df_transformed.to_csv(OUTPUT_SINGLE_ROW, index=False)    